In [1]:
# Notebook to showcase the use of XGBoost for paramterization of ECMS as mentioned in the supplementary information of the paper. 

In [ ]:
df = preprocess_data(d_path + "train_data.csv")
df_test= preprocess_data(pred_dfile)

In [ ]:
df["param_strs"] = df.apply(
    lambda x: process_batch_element_params_str(x.Parameters), axis=1
)
df["param_values"] = df.apply(
    lambda x: process_batch_element_params(x.Parameters), axis=1
)

## Unwrap data frame for tsfresh
df_ts = unwrap_df(df)

for ecm in df["Circuit"].unique():
    print(f"Fitting regression for {ecm}")

    ## Subselect circuits of given ECM
    df_ecm = df[df["Circuit"] == ecm]
    df_ts_ = df_ts[df_ts["id"].isin(df_ecm.index)]

    df_x = pd.DataFrame(index=np.unique(df_ts_["id"]))
    df_y = pd.DataFrame(
        df_ecm["param_values"].to_list(),
        columns=df_ecm["param_strs"].loc[df_ecm.index[0]]
    )

    print("Starting regr. feature extraction and training pipeline")
    pipe = create_pipeline_reg(df_ts)
    ## Evaluate features and fit XGBoost Model
    pipe.fit(df_x, df_y)

    print("trainign completed, writing model and predictions to disc")
    dump(pipe, f"{m_path}reg/xgb-ts-feat-reg-{ecm}-data-pipeline.joblib")

    with open(f"{m_path}{ecm}_param-names.txt", "w") as f:
        json.dump(df_ecm["param_strs"].loc[df_ecm.index[0]].tolist(), f)